In [1]:
import ROOT

Welcome to ROOTaaS 6.06/06


In [2]:
%%cpp
// cpp imports
#include <cstdlib>
#include <vector>
#include <iostream>
#include <map>
#include <string>

#include "TFile.h"
#include "TTree.h"
#include "TString.h"
#include "TSystem.h"
#include "TROOT.h"
#include "TStopwatch.h"

#include "TMVA/Tools.h"
#include "TMVA/Reader.h"
// our vectors, which we are using to get around the fundamental problem, that rootpy can't construct root/cpp floats
// and we therefore otherwiese couldn't give a reference over to tmva in the later code
std::vector<float> vars;
std::vector<TString> varsName;

In [3]:
# get metadata from the original trainingset
originalTrainingSet = "/root/Downloads/dataSet_CERN_to_RAL.root"
inputFile = ROOT.TFile.Open( originalTrainingSet )
regTree = inputFile.Get("TreeR")
leaves = regTree.GetListOfLeaves()
varsName = []
# add all variables, except throughput and timestamp
for i in xrange(len(leaves)):
    if (leaves[i].GetName() == "timestamp") or (leaves[i].GetName() == "throughput"):
            continue
    ROOT.varsName.push_back(leaves[i].GetName())
    # keep a copy in python which is easyer to handle
    varsName.append(leaves[i].GetName())
# resize other vector to fit
ROOT.vars.resize(len(ROOT.varsName))

In [4]:
%%cpp
// switch to C++ to avoid the annoying reference problem, mentioned above
// --- Create the Reader object
TMVA::Reader *reader = new TMVA::Reader( "!Color:!Silent" );

// Create a set of variables and declare them to the reader
// - the variable names MUST corresponds in name and type to those given in the weight file(s) used
// add the variables that we just stored in our C++ vector (what a time to be alive)

for (int i = 0; i < varsName.size(); i++){
    reader->AddVariable( varsName[i], &vars[i] );
}

In [5]:
methodName = "MLP method"
#weightfile = "/root/Downloads/buildDirRoot/tutorials/tmva/weights/TMVARegression_MLP.weights.xml"
weightfile = "/root/Downloads/weights/TMVARegression_MLP.weights.xml"
ROOT.reader.BookMVA( methodName, weightfile )

<ROOT.TMVA::MethodMLP object ("MLP") at 0x55f71e0>

--- Reader                   : Booking "MLP method" of type "MLP" from /root/Downloads/weights/TMVARegression_MLP.weights.xml.
--- MethodBase               : Reading weight file: /root/Downloads/weights/TMVARegression_MLP.weights.xml
--- MLP                      : Read method "MLP" of type "MLP"
--- MLP                      : MVA method was trained with TMVA Version: 4.2.1
--- MLP                      : MVA method was trained with ROOT Version: 6.06/06
--- DataSetInfo              : Added class "Regression"	 with internal class number 0
--- MLP                      : Building Network
--- MLP                      : Initializing weights
--- Reader                   : Booked classifier "MLP" of type: "MLP"


In [6]:
for i in xrange(len(ROOT.varsName)):
    ROOT.vars[i] = i

In [7]:
ROOT.reader.EvaluateRegression( "MLP method" )[0]

1.200074553489685

In [10]:
ROOT.vars[2]

2.0